In [1]:
# 四半期の騰落率が大きく動いた銘柄の過去分析行う

In [2]:
# 読み込みファイルパスの設定とimportしたいmoduleパス(pythonパス)の設定
from pathlib import Path
import os, sys

CURRENT_DIR = Path(os.getcwd())
PJ_DIR = CURRENT_DIR.parent.parent
LIB_DIR = PJ_DIR / "lib"
DATA_DIR = PJ_DIR / "data" 

sys.path.append(str(LIB_DIR))

# notebook内で利用するmoduleのimport
from lib_dataprocess import KessanPl, PricelistFig, PricelistPl, MeigaralistPl, read_data
from lib_dataprocess import ShikihoPl, FinancequotePl, print_finance_quote, PricelistFig, KessanFig
import polars as pl
from datetime import date
from dateutil.relativedelta import relativedelta

fp = DATA_DIR/"reviced_pricelist.parquet"
df = read_data(fp)
RevPl = PricelistPl(df)

# dataの読み込み
fp = DATA_DIR/"raw_pricelist.parquet"
df = read_data(fp)
PPL = PricelistPl(df)

fp = DATA_DIR/"finance_quote.parquet"
df = read_data(fp)
FPL = FinancequotePl(df)
FPL.with_columns_market_cap()

fp = DATA_DIR/"meigaralist.parquet"
df = read_data(fp)
MPL = MeigaralistPl(df)

fp = DATA_DIR/"shikiho.parquet"
df = read_data(fp)
SPL = ShikihoPl(df)

In [3]:
#####
##### KessanPl.dfの加工
#####

In [4]:
#パラメータのセット
# 決算日の指定 -> 指定した日の四半期決算でランキングを作成する
valuation_date = date.today()
valuation_date = date(2024, 9, 26)
settlement_type = "四"
pricelist_df = RevPl.df

In [31]:
# 読み込みと加工
KPL = KessanPl()
df = KPL.get_settlement_performance(settlement_type, pricelist_df)
KPL = KessanPl(df)
df = KPL.get_latest_quater_settlements(valuation_date)
# 売上高25億以上
df = df.filter(pl.col("sales")>=2500)

# sortして出力
df = df.sort(by=["price_change_rate"], descending=[True])
kpl_df = df
kpl_df

code,announcement_date,settlement_date,settlement_type,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater,price_change_rate
i64,date,date,str,i64,i64,i64,i64,f64,f64,i64,f64
4935,2024-08-09,2024-06-30,"""四""",2529,144,139,81,13.7,5.7,2,159.1
6557,2024-07-31,2024-06-30,"""四""",3009,116,111,203,65.8,3.9,1,124.7
4348,2024-07-31,2024-06-30,"""四""",20862,1282,1311,831,15.1,6.1,1,124.6
6704,2024-08-09,2024-06-30,"""四""",4572,-822,-800,-815,-54.7,-18.0,1,99.6
6927,2024-08-06,2024-06-30,"""四""",3232,698,721,509,28.1,21.6,1,78.1
…,…,…,…,…,…,…,…,…,…,…,…
4761,2024-07-30,2024-06-30,"""四""",4685,4,49,31,2.9,0.1,1,-48.2
5255,2024-08-14,2024-06-30,"""四""",2542,-6099,-5505,-5554,-161.8,-239.9,2,-51.5
6254,2024-08-09,2024-06-30,"""四""",7629,362,494,411,11.0,4.7,1,-51.9


In [62]:
kpl_df[0:10]

code,announcement_date,settlement_date,settlement_type,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater,price_change_rate
i64,date,date,str,i64,i64,i64,i64,f64,f64,i64,f64
4935,2024-08-09,2024-06-30,"""四""",2529,144,139,81,13.7,5.7,2,159.1
6557,2024-07-31,2024-06-30,"""四""",3009,116,111,203,65.8,3.9,1,124.7
4348,2024-07-31,2024-06-30,"""四""",20862,1282,1311,831,15.1,6.1,1,124.6
6704,2024-08-09,2024-06-30,"""四""",4572,-822,-800,-815,-54.7,-18.0,1,99.6
6927,2024-08-06,2024-06-30,"""四""",3232,698,721,509,28.1,21.6,1,78.1
4490,2024-07-12,2024-05-31,"""四""",2520,385,378,60,-0.7,15.3,1,64.8
9067,2024-08-07,2024-06-30,"""四""",11307,243,292,187,6.5,2.1,1,60.6
3738,2024-08-01,2024-06-30,"""四""",104750,3097,4284,2805,50.3,3.0,1,57.4
5010,2024-08-14,2024-06-30,"""四""",5864,799,685,637,32.3,13.6,2,54.6


In [6]:
#####
##### 最新ファンダメンタルズ&業績情報
#####

In [69]:
# parametor
code = 9067

In [70]:
# ファンダメンタルズ
print_finance_quote(PPL.df, FPL.df, code, valuation_date=valuation_date)
print()
SPL.print_stock_df(code, num=4, valuation_date=valuation_date)

丸運(9067)の銘柄情報

終値: 501.0円(2024年09月26日)
予想配当利回り: 2.0%(2024年09月26日)
予想PER: 25.39倍(2024年09月26日)
実績PBR: 0.58倍(2024年09月26日)
自己資本比率: 67.1%(2024年09月26日)
予想ROE: 2.28%(2024年09月26日)
予想ROA: 1.53%(2024年09月26日)
時価総額: 145.1億円(2024年09月26日)

9067(丸運)の四季報データ履歴

発行日: 2024年09月13日
【連続増配】
  貨物輸送は数量横ばいも値上げ浸透。エネルギー輸送も石油の数量減響くが、外注費上昇を受けた運賃改定により伸長。労務費・物流費増こなし営業益続伸。有価証券売却益(約2.5億円)。連続増配。
【提携】
  ベトナム子会社が同国の梱包専門会社の株式約40%を取得。梱包から保管、輸送まで一貫サービス提供へ。需要増にらみ、国内の危険物倉庫の増強を鋭意検討。

発行日: 2024年06月14日
【続伸】
  エネルギー輸送は数量減に加え労務費上昇で軟調。が、貨物輸送は素材物流軸に数量回復、前期価格改定もフル寄与。貨物・海外物流で一貫物流業務の新規業務獲得が上乗せ。営業利益続伸。連続増配。
【営業強化】
  4月貨物輸送事業部に営業開発部を設置、営業情報を全社で共有するシステムも稼働し、新規業務受注強化へ。24年3月川崎市東扇島の毒劇物倉庫が稼働。

発行日: 2024年03月16日
【連続増益】
  貨物は素材物流の数量減響く。エネルギーは暖冬で石油輸送想定以下。ただ基幹システム導入費減で営業増益。25年3月期はエネルギー輸送横ばい。が、貨物輸送は素材物流軸に数量回復。前期価格改定も定着。増益続く。連続増配余地。
【物流問題】
  24年物流問題を受け、鉄道輸送など通運事業への取り組み強化。危険物輸送の需要増にらみ、倉庫増強など検討。

発行日: 2023年12月15日
【増配】
  貨物輸送は素材物流など数量減に加え、航空・海上運賃下落響く。エネルギー輸送も回復緩慢。ただ価格改定が浸透。基幹システム導入費用減や一貫物流での費用抑制効き、営業益は反転増。増配。25年3月期は貨物輸送の取扱数量増える。
【還元】
  24年3月期から連結配当性向50%

In [71]:
# 業績推移
KFIG = KessanFig(code, "四半期", end_settlement_date=valuation_date)
KFIG.add_trace_profits()
KFIG.fig.show()

In [72]:
#####
##### 株価推移
#####

In [73]:
# para
delta = relativedelta(days=100)

In [74]:
# 日足
df = kpl_df.filter(pl.col("code")==code)
ann_date = df["announcement_date"].to_list()[-1]
print(f'決算発表日: {ann_date}')
start_date = ann_date - delta
end_date = ann_date + delta

PFIG = PricelistFig(code, RevPl.df, MPL.df, start_date, end_date)
# 決算発表日にvlineを引く
PFIG.add_vline_announcement_date()
PFIG.fig.show()

決算発表日: 2024-08-07
